In [30]:
import pandas as pd
from pandas import option_context
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score,classification_report

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

from collections import Counter

from textblob import TextBlob

In [33]:
stop = stopwords.words('english')

## Sentiment analysis with random forrest

In [71]:
df = pd.read_csv('training_base_sent.csv')
df.dropna(inplace=True)

In [72]:
def remove_space(text):
    text = text.strip()
    text = re.sub("\s\s+", " ", text)
    return text
    
df['tweet_compound'] = df.tweet_compound.map(remove_space)
df['tweet'] = df.tweet.map(remove_space)

In [73]:
tr, test = train_test_split(df,test_size = 0.2,random_state = 10)

In [74]:
tr.groupby('sentiment').count()

,Unnamed: 0,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound
sentiment,,,,,,,,,,
-1.0,1391,1391,1391,1391,1391,1391,1391,1391,1391,1391
0.0,874,874,874,874,874,874,874,874,874,874
1.0,324,324,324,324,324,324,324,324,324,324


In [75]:
# creating bag of words model 
cv = CountVectorizer(max_features=1000) 
  
X = cv.fit_transform(tr['tweet']).toarray() 
y = tr[['sentiment']].iloc[:, 0].values 

In [76]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X,y)

X_test = cv.fit_transform(test['tweet']).toarray() 
y_pred=clf.predict(X_test)

In [77]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(test['sentiment'], y_pred))

Accuracy: 0.5138888888888888


In [78]:
confusion_matrix(test['sentiment'], y_pred)

array([[272,  42,  17],
       [161,  53,  13],
       [ 68,  14,   8]])

In [79]:
print(classification_report(test['sentiment'], y_pred))

              precision    recall  f1-score   support

        -1.0       0.54      0.82      0.65       331
         0.0       0.49      0.23      0.32       227
         1.0       0.21      0.09      0.12        90

    accuracy                           0.51       648
   macro avg       0.41      0.38      0.36       648
weighted avg       0.48      0.51      0.46       648

